In [1]:
# Imports and load dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb; sb.set()
planets = sb.load_dataset('planets')
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [2]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


# `groupby()`
## General Concepts
1. It follows **split-apply-combine** procedure. But `grouby()` abstracts this process by using **lazy evaluation**. That's why I misunderstood `grouby()` as a new class. It is actually just a list of df, just that it is not yet executed for efficiency purposes
  
## Operations
1. Column Indexing: instead of `df[[groupcol, col1]].groupby().sum()`, I can directly `groupby()` first before extracting the column!
2. Iteration over groups: Since `groupby()` is just lazy, actually I can extract `(groupcol, group)` and start iterating!
    - `group` is a Series or df
3. Dispatch methods: with split-apply-combine, I can apply functions on each individual group
4. Aggregate: **reduce shape** of df or Series. 
> - `aggregate({cols: functions})`
> - `aggregate([functions])` for symmetrical table
4. Filter: `.filter(func)`
> - `def func(df_group)`: returns a bool
5. Transform: **retains shape** of df or Series
> - `.transform(normally lambda func)`
6. Apply: `.apply(func)` custom functions (different from `.aggregate()`)
> - `def func(df)`: returns series/df/scalar

In [3]:
# 1. Column indexing
planets.groupby("method")['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [4]:
# 2. Iteration over groups
for method, group in planets.groupby("method"):
    print(f"{method:30s} shape={group.shape}")

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [5]:
# 3. Dispatch methods
planets.groupby("method")['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


In [6]:
# 4. aggregate to produce SYMMETRICAL table
df.groupby("key").aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [7]:
# 4b. aggregate each column with a DIFFERENT function
df.groupby("key").aggregate({'data1': min, 'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [8]:
# 5. Filtering
def filter_func(x):  # returns bool or list of bool
    return x['data2'].std() > 4

df.groupby('key').filter(filter_func)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


In [19]:
# 6. Transformation
df.groupby("key").transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [17]:
# 7. Apply
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,-1.5,1.0
1,B,-1.5,-3.5
2,C,-1.5,-3.0
3,A,1.5,-1.0
4,B,1.5,3.5
5,C,1.5,3.0


# Specify Split Key
1. `groupby(list)`: if I want to customise the group members
2. `groupby(dict)`: if I want to group by **index**, but change the group name
3. `groupby(func)`: if I want to group by **index**, but slightly change the group name with a function

In [11]:
# 1. groupby(list)
L = [0, 1, 0, 1, 2, 0]
display(df.groupby(L).sum())
display(df.groupby(df['key']).sum()) # verbose way

,data1,data2
0,7,17
1,4,3
2,4,7


,data1,data2
key,,
A,3,8
B,5,7
C,7,12


In [12]:
# 2. groupby(dict)
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
df2.groupby(mapping).sum()

,data1,data2
key,,
consonant,12,19
vowel,3,8


In [13]:
# 3. Python function
df2.groupby(str.lower).mean()

,data1,data2
key,,
a,1.5,4.0
b,2.5,3.5
c,3.5,6.0


In [14]:
decade = 10*(planets['year']//10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0
